In [2]:
# FuelHH Stream seems to be more complete data, includes interconnectors, average MW value over 30 mins (also available in 5 min intervals - inst). 
# This code also pulls wind and solar data.
import requests
import pandas as pd
from datetime import date, timedelta
import os

# Set your date range
start_date = date(2024, 7, 1)
end_date = date(2025, 6, 30)

# -----------------------------------
# Fetch FUELHH data
# -----------------------------------
fuelhh_url = "https://data.elexon.co.uk/bmrs/api/v1/datasets/FUELHH/stream"
fuelhh_params = {
    "settlementDateFrom": start_date.strftime("%Y-%m-%d"),
    "settlementDateTo": end_date.strftime("%Y-%m-%d"),
    "format": "json"
}
fuelhh_resp = requests.get(fuelhh_url, params=fuelhh_params)
fuelhh_resp.raise_for_status()
fuelhh_data = fuelhh_resp.json()
fuelhh_df = pd.DataFrame(fuelhh_data)

# Keep only needed columns
fuelhh_df = fuelhh_df[['settlementDate', 'settlementPeriod', 'fuelType', 'generation']]
fuelhh_df['settlementDate'] = pd.to_datetime(fuelhh_df['settlementDate'])

# Pivot fuel types
pivot_fuelhh = fuelhh_df.pivot_table(
    index=['settlementDate', 'settlementPeriod'],
    columns='fuelType',
    values='generation',
    aggfunc='sum'
)

pivot_fuelhh.columns = [f"{col} (Average MW)" for col in pivot_fuelhh.columns]
pivot_fuelhh = pivot_fuelhh.reset_index()

# -----------------------------------
# Fetch AGWS data (Wind & Solar)
# -----------------------------------
agws_url = "https://data.elexon.co.uk/bmrs/api/v1/datasets/AGWS/stream"
agws_params = {
    "publishDateTimeFrom": start_date.strftime("%Y-%m-%d"),
    "publishDateTimeTo": (end_date + timedelta(days=2)).strftime("%Y-%m-%d"),
    "format": "json"
}
agws_resp = requests.get(agws_url, params=agws_params)
agws_resp.raise_for_status()
agws_data = agws_resp.json()
agws_df = pd.DataFrame(agws_data)

# Use corrected column names
agws_df = agws_df[['settlementDate', 'settlementPeriod', 'psrType', 'quantity']]
agws_df['settlementDate'] = pd.to_datetime(agws_df['settlementDate'])

# Pivot psrTypes (e.g., Wind, Solar)
pivot_agws = agws_df.pivot_table(
    index=['settlementDate', 'settlementPeriod'],
    columns='psrType',
    values='quantity',
    aggfunc='sum'
)

pivot_agws.columns = [f"{col} (Average MW)" for col in pivot_agws.columns]
pivot_agws = pivot_agws.reset_index()

# -----------------------------------
# Merge FUELHH + AGWS (left join: only keep rows present in FUELHH)
# -----------------------------------
combined_df = pd.merge(pivot_fuelhh, pivot_agws, on=['settlementDate', 'settlementPeriod'], how='left')
combined_df = combined_df.sort_values(['settlementDate', 'settlementPeriod'])

# -----------------------------------
# Save to CSV
# -----------------------------------
start_str = start_date.strftime("%Y-%m-%d")
end_str = end_date.strftime("%Y-%m-%d")
filename = f"FUELHH_AGWS_{start_str}_to_{end_str}.csv"

output_dir = r"C:\Users\spice\Dropbox\Documents\Imperial 2024.2025\MECH70038 - Research Projects\_My Thesis\Data"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, filename)

combined_df.to_csv(output_file, index=False)
print(f"Saved to {output_file}")



Saved to C:\Users\spice\Dropbox\Documents\Imperial 2024.2025\MECH70038 - Research Projects\_My Thesis\Data\FUELHH_AGWS_2024-07-01_to_2025-06-30.csv
